In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [52]:
# Load the data

data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [53]:
# Data preprocessing
# Drop irrelevant columns

data = data.drop(["RowNumber","CustomerId", "Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [54]:
# Encoding categorical features

label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [55]:
# OneHotEncoding the "Geography" column

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

geo_encoded = ohe.fit_transform(data[["Geography"]])

geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [56]:
ohe.get_feature_names_out(["Geography"])


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [57]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=ohe.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [58]:
# Combine one hot encoded columns with the original data

data = pd.concat([data.drop(["Geography"], axis=1), geo_encoded_df], axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [59]:
# save the encoders

with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("ohe_geo.pkl", "wb") as file:
    pickle.dump(ohe,file)
    

In [60]:
# Devide the dataset into indepadant and depandent features

X = data.drop(["Exited"],axis=1)
y = data["Exited"]

# Splitting the data into training and testing set

X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale the features
stdsclr = StandardScaler()

X_train = stdsclr.fit_transform(X_train)
X_test = stdsclr.transform(X_test)

In [61]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [62]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [63]:
# save the standard scaler 

with open("stdsclr.pkl", "wb") as file:
    pickle.dump(stdsclr, file)

## ANN Training

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


## ANN MODEL TRAINING

In [65]:
model = Sequential([
    Dense(64,activation="relu", input_shape=(X_train.shape[1],)), # hidden layer 1 connected with input layer
    Dense(32, activation="relu"), # hidden layer 2
    Dense(1,activation="sigmoid") # output layer
])

In [66]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## COMPILING MODEL

In [67]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

## SETTIG UP TENSORBOARD

In [68]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)

## SET UP EARLY STOPPING

In [69]:
early_Stopping_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

## TRAINING THE MODEL

In [70]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=100, callbacks=[tensorflow_callbacks,early_Stopping_callback])

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3985 - accuracy: 0.8365 - val_loss: 0.3624 - val_accuracy: 0.8535
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3544 - accuracy: 0.8570 - val_loss: 0.3554 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3468 - accuracy: 0.8555 - val_loss: 0.3471 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3435 - accuracy: 0.8596 - val_loss: 0.3466 - val_accuracy: 0.8625
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3389 - accuracy: 0.8626 - val_loss: 0.3552 - val_accuracy: 0.8550
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3393 - accuracy: 0.8626 - val_loss: 0.3438 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3352 - accuracy: 0.8624 - val_loss: 0.3443 - val_accuracy: 0.8635

In [71]:
model.save("model.h5")

j:\data_science_projects\deep learning\ANN\annclassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## LOADING TENSORBOARD EXTENSION

In [72]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [73]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 12028), started 2 days, 0:01:27 ago. (Use '!kill 12028' to kill it.)